In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/irina/.secret/yelp_api.json') as f:
    login = json.load(f)

In [3]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [ ]:
LOCATION = 'Oakland, CA'
TERM = 'thai food'

In [6]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == True:
        
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")        
        folder = os.path.dirname(JSON_FILE)
        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [7]:
create_json_file(JSON_FILE, delete_if_exists=True)
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
total_results = results['total']
results_per_page = len(results['businesses'])
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_oakland_thai.json already exists. Deleting previous file...
[i] Data/results_in_progress_oakland_thai.json not found. Saving empty list to new file.
- 0 previous results found.


90

In [8]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/90 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [9]:
oakland_thai = pd.read_json(JSON_FILE)
display(oakland_thai.head(), oakland_thai.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,cgImAqtrpVg7hN_6lsFJrQ,farmhouse-kitchen-thai-cuisine-oakland,Farmhouse Kitchen Thai Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/htkVpf...,False,https://www.yelp.com/biz/farmhouse-kitchen-tha...,1988,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.794215, 'longitude': -122.275465}","[delivery, restaurant_reservation]",$$$,"{'address1': '336 Water St', 'address2': '', '...",+15104190541,(510) 419-0541,4140.003330
1,Jlt5w5ItA8ugiXif2VVQ5g,champa-garden-oakland,Champa Garden,https://s3-media1.fl.yelpcdn.com/bphoto/sAclWP...,False,https://www.yelp.com/biz/champa-garden-oakland...,2223,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.7981399, 'longitude': -122.2446}","[delivery, pickup]",$$,"{'address1': '2102 8th Ave', 'address2': '', '...",+15102388819,(510) 238-8819,1569.633720
2,Cfk_zVxh8mwLdFbKGoURCg,thai-taste-kitchen-oakland,Thai Taste Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/y7tAg1...,False,https://www.yelp.com/biz/thai-taste-kitchen-oa...,232,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 37.8241203332419, 'longitude': -1...",[delivery],$$,"{'address1': '3814 Piedmont Ave', 'address2': ...",+15103507542,(510) 350-7542,4254.036333
3,CVaa6r_xMJTBUoTk3whjhw,wawa-thai-food-oakland,Wawa Thai Food,https://s3-media2.fl.yelpcdn.com/bphoto/dV_GkV...,False,https://www.yelp.com/biz/wawa-thai-food-oaklan...,191,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 37.7971694, 'longitude': -122.204...","[delivery, pickup]",$$,"{'address1': '3009 Macarthur Blvd', 'address2'...",+15104793465,(510) 479-3465,2149.873722
4,1vJLE77VZw0dukHd5bw35A,daughter-thai-kitchen-oakland,Daughter Thai Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/rxUsK3...,False,https://www.yelp.com/biz/daughter-thai-kitchen...,1490,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 37.82693496, 'longitude': -122.21...","[delivery, restaurant_reservation]",$$,"{'address1': '6118 Medau Pl', 'address2': '', ...",+15108232354,(510) 823-2354,4167.237104


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,-yUSIgFH9CTMekEfAPKeVA,hunan-villa-pinole,Hunan Villa,https://s3-media1.fl.yelpcdn.com/bphoto/kaKG1b...,False,https://www.yelp.com/biz/hunan-villa-pinole?ad...,332,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 38.003969299271, 'longitude': -12...",[delivery],$$,"{'address1': '632 San Pablo Ave', 'address2': ...",+15107241368,(510) 724-1368,24723.850418
996,S2v_9CKwsEyMVpTbrlewGw,king-dong-chinese-berkeley,King Dong Chinese,https://s3-media1.fl.yelpcdn.com/bphoto/6HUhQo...,False,https://www.yelp.com/biz/king-dong-chinese-ber...,526,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 37.86526, 'longitude': -122.267218}","[pickup, delivery]",$$,"{'address1': '2429 Shattuck Ave', 'address2': ...",+15108416196,(510) 841-6196,8806.063354
997,4De0iMwSPF2aJ_QGTye4TQ,nan-king-china-bistro-san-francisco,Nan King China Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/knJuIE...,False,https://www.yelp.com/biz/nan-king-china-bistro...,615,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 37.76291, 'longitude': -122.4659}","[pickup, delivery]",$$,"{'address1': '1360 9th Ave', 'address2': '', '...",+14157532900,(415) 753-2900,21134.309642
998,1ErPhzdCaoMVSHsXc9TOmQ,310-eatery-albany,310 Eatery,https://s3-media2.fl.yelpcdn.com/bphoto/RqNHGe...,False,https://www.yelp.com/biz/310-eatery-albany?adj...,1055,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 37.89241, 'longitude': -122.29927}","[pickup, delivery]",$$,"{'address1': '747 San Pablo Ave', 'address2': ...",+15106973878,(510) 697-3878,12757.498899
999,BbmiVo55an8Ps8RekommgQ,sing-siu-zai-min-san-francisco,Sing Siu Zai Min,https://s3-media2.fl.yelpcdn.com/bphoto/cgDmvK...,False,https://www.yelp.com/biz/sing-siu-zai-min-san-...,44,"[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.5,"{'latitude': 37.7376, 'longitude': -122.46928}","[pickup, restaurant_reservation, delivery]",NaN,"{'address1': '321 W Portal Ave', 'address2': '...",+14155718340,(415) 571-8340,22029.107122


In [11]:
oakland_thai.duplicated(subset='id').sum()

0

In [12]:
# save the final results to a compressed csv
oakland_thai.to_csv('Data/final_results_oakland_thai.csv.gz', compression='gzip',index=False)